# Single Event with Complex Analysis

## Part Three (Streaming Analysis)

#### Prerequisites
This notebook is designed to work with a Stroom server process running on `localhost`, into which data from `EventGen` application has been ingested and indexed in the manner described in `stroom-analytic-demo`.

You must set the environmental variable `STROOM_API_KEY` to the API token associated with a suitably privileged Stroom user account before starting the Jupyter notebook server process.

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col, coalesce, unix_timestamp,lit,to_timestamp,hour,date_format,date_trunc,window
from pyspark.ml.feature import OneHotEncoderEstimator,VectorAssembler,StringIndexer
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel
from pyspark.ml import Pipeline,PipelineModel
from IPython.display import display
import time,os

#### Schema Discovery
It is necessary to specify the structure of the JSON data arriving on the topic.  This structure can be determined at runtime.

As the same format of data is also available via an indexed search using the `stroom-spark-datasource`, one way to determine the JSON schema is by interrogating the data held in the `Sample Index` Stroom index.

The specified pipeline is a Stroom Search Extraction Pipeline that uses the stroom:json XSLT function to create a JSON representation of the entire event.  This field is called "Json" by default but the name of the field that contains the JSON representation can (optionally) be changed with the parameter jsonField.

In this manner, all data is returned as a single JSON structure within the field **json**

In [2]:
spark = SparkSession \
    .builder \
    .appName("MyTestApp") \
    .getOrCreate()
schemaDf = spark.read.format('stroom.spark.datasource.StroomDataSource').load(
        token=os.environ['STROOM_API_KEY'],host='localhost',protocol='http',
        uri='api/stroom-index/v2',traceLevel="0",
        index='32dfd401-ee11-49b9-84c9-88c3d3f68dc2',pipeline='13143179-b494-4146-ac4b-9a6010cada89',
        maxResults='300000').filter((col('idxEventTime') > '2018-01-01T00:00:00.000Z')
            & (col('idxEventTime') < '2018-01-02T00:00:00.000Z'))

print ('Using ', schemaDf.count(), ' records for schema discovery.')
json_schema = spark.read.json(schemaDf.rdd.map(lambda row: row.json)).schema

json_schema

Using  3465  records for schema discovery.


StructType(List(StructField(EventDetail,StructType(List(StructField(Authenticate,StructType(List(StructField(Action,StringType,true),StructField(Outcome,StructType(List(StructField(Permitted,StringType,true),StructField(Reason,StringType,true),StructField(Success,StringType,true))),true),StructField(User,StructType(List(StructField(Id,StringType,true))),true))),true),StructField(Process,StructType(List(StructField(Action,StringType,true),StructField(Command,StringType,true),StructField(Type,StringType,true))),true),StructField(TypeId,StringType,true))),true),StructField(EventId,StringType,true),StructField(EventSource,StructType(List(StructField(Client,StructType(List(StructField(HostName,StringType,true))),true),StructField(Device,StructType(List(StructField(HostName,StringType,true))),true),StructField(Generator,StringType,true),StructField(System,StructType(List(StructField(Environment,StringType,true),StructField(Name,StringType,true))),true),StructField(User,StructType(List(Struct

## Spark Dataframe for streaming analysis
Create a Spark DF driven via the Kafka topic

In [3]:

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "ANALYTIC-DEMO-UEBA") \
  .option("startingoffsets", "earliest")\
  .load()

## Two ways to work with windows
It is possible to create the windowed counts using window functions using the dataframe function `over` or aggregate functions (grouped by timestamp).  The first of these calculates a new value for every record, and therefore allows rolling windowed operations to be performed.

The aggregate function approach follows.

In [6]:
def process_batch(df, epoch_id):
    df.sort(col('count').desc()).show()
    #df.filter(col('count') > col('prediction') * 2).show()
  

In [ ]:
#window = Window.orderBy("timestamp").partitionBy()

wideDf = df.withColumn('json',col('value').cast('string')).\
    withColumn('evt', from_json(col('json'), json_schema)).\
    withColumn ('timestamp', to_timestamp(col('evt.EventTime.TimeCreated')).cast("timestamp")).\
    withColumn('operation', col('evt.EventDetail.TypeId')).\
    filter(col('operation') == 'Authentication Failure').\
    withColumn('streamid', col('evt.StreamId')).\
    withColumn('eventid', col('evt.EventId')).\
    withColumn('day', date_format('timestamp', 'EEEE')).\
    withColumn('hour', hour("timestamp").alias("hour")).\
    dropDuplicates(["eventid", "streamid"]).\
    groupBy(window ("timestamp", "1 hour"),'day','hour','operation').count()

pipelineModel = PipelineModel.load("models/inputVecPipelineModel")

# lrModel.save("lrAuthFailuresModel")

# hourEncoder = OneHotEncoderEstimator(inputCols=['hour'],outputCols=['hourVec'])
# dayNameIndexer = StringIndexer(inputCol="day",outputCol="dayCat")
# dayEncoder = OneHotEncoderEstimator(inputCols=['dayCat'],outputCols=['dayVec'])
# basicPipeline = Pipeline(stages=[hourEncoder, dayNameIndexer, dayEncoder])

# vecDf = basicPipeline.fit(wideDf).transform(wideDf)

# vectorAssembler = VectorAssembler(inputCols = ['hourVec','dayVec'], outputCol = 'features')

# featuresDf = vectorAssembler.transform(vecDf).select('features','count')

featuresDf = pipelineModel.transform(wideDf)

vectorAssembler = VectorAssembler(inputCols = ['hourVec','dayVec'], outputCol = 'features')

fullDf = vectorAssembler.transform(featuresDf).select('window','features','count')

lrModel = LogisticRegressionModel.load("models/logisticRegressionAuthFailuresModel")

lrDf = lrModel.transform (fullDf)

#outputMode can be append, complete or update
query = lrDf.writeStream.\
    foreachBatch (process_batch).\
    outputMode("complete").\
    start()

query.awaitTermination()

 


+--------------------+--------------------+-----+--------------------+--------------------+----------+
|              window|            features|count|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|[2019-11-05 14:00...|(29,[14,25],[1.0,...|   25|[-4.0702780504231...|[1.45000719330590...|      21.0|
|[2019-11-04 13:00...|(29,[13,27],[1.0,...|   24|[-4.0819635222495...|[1.58244653249414...|      18.0|
|[2019-11-04 16:00...|(29,[16,27],[1.0,...|   23|[-4.0743370372498...|[1.26650973752681...|      19.0|
|[2019-11-05 13:00...|(29,[13,25],[1.0,...|   23|[-4.0739692576681...|[1.58899507436920...|      20.0|
|[2019-11-05 11:00...|(29,[11,25],[1.0,...|   22|[-4.0583778204207...|[1.01793374247146...|      16.0|
|[2019-11-05 12:00...|(29,[12,25],[1.0,...|   22|[-4.0681788075552...|[1.33565068408936...|      19.0|
|[2019-11-01 15:00...|(29,[15,24],[1.0,...|   22|[-4.0693609270898...|[1.

We can now create the entire feature vector.  Shown below with what will be the required output vector (actually a simple scalar "count")